# Wrangling Open Street Map into MongoDB
#### Map Area
Bellingham, WA, USA (greater area for minimum size requirements)

Extract pulled from openstreetmap.org using <a href="https://www.openstreetmap.org/export">Overpass API</a>.
```
<bounds minlat="48.6020000" minlon="-122.8244000" maxlat="49.0027000" maxlon="-122.0787000"/>
```
<a id="contents"></a>
## Contents
<ul>
    <li><a href="#initStats">Document Structure and Initial Stats</a></li>
    <li><a href="#tagAudits">Sample Tag Audits</a></li>
    <li><a href="#trouble">Trouble in the Data</a></li>
    <li><a href="#load">Cleanup, Write to JSON, Load into MongoDB</a></li>
    <li><a href="#mongoStats">Import and MongoDB Stats</a></li>
    <li><a href="#mongoAudit">Auditing in Mongo</a></li>
    <li><a href="#finalQ">Final Queries: using lookup; another audit; indexes</a></li>
    <li><a href="#recommendation">A Parting Recommendation</a>
</ul>

<a id="initStats"></a>
## Document Structure and Initial Stats
First, we'll look at the expected document and database structures before and after. After that, we can dive into the tags, and make a cleanup plan for some of them.

To build our JSON document, we skip the outer tags ("osm", "note", "meta", and "bounds") and focus on the three main elements ("node", "way", and "relation") for top-level documents, loading them into a single MongoDB collection. Their attributes and subelements ("tag", "nd", and "member") are their subdocuments.

Element creation attributes ("changeset", "timestamp", "uid", "user", and "version") become keys in one subdocument called "created". Node lattitude and longitude form a single positional vector array under the "pos" key. The "ref" attribute of the "nd" subelements similarly form a list under the "node_refs" key of way documents. Relation elements' "member" subelements go into a list of subdocuments with their attribute keys ("re", "role", "type").

While tag elements only have "k" and "v" attributes, they have the messiest range of possible values that pose structural challenges as well as copy editing challenges. This is where we focus our cleanup efforts. Unlike other elements, we drop tag attribute keys, and use their values as keys and values in the JSON document.

Okay, let's get a bird's eye view of the document. Since we're cleaning up before loading into MongoDB, we'll just make some quick and dirty pandas dataframes for this initial analyis. One thing worth noting is that nodes are the most common top-level element, which the nd element is the most common element, which means some nodes get referenced more than once.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import re
import xml.etree.ElementTree as ET
import pymongo
from pymongo import MongoClient
import os

# My modules.
import osm_structure_audit
import clean_and_write
import mongo_audit

filename = "greater_bellingham.osm"

list_query = lambda cursor: [doc for doc in cursor]

In [2]:
el_df, tag_df = osm_structure_audit.get_eldf_tagdf(filename)

# iter() stops parsing after 105 subelements on my computer.
# Consequently, it won't get to way and relation elements within osm.
# I'll add them here.
el_df.loc["osm", "sub_els"].add("way")
el_df.loc["osm", "sub_els"].add("relation")
el_df
print("The set of node attributes get cut off in the output:")
el_df.loc["node", "attributes"]

,count,sub_els,attributes
element_type,,,
osm,1,"{way, note, meta, tag, node, bounds, relation}","{version, generator}"
note,1,{},{}
meta,1,{},{osm_base}
bounds,1,{},"{maxlat, minlat, minlon, maxlon}"
node,355044,{tag},"{version, timestamp, uid, lat, id, lon, change..."
tag,131881,{},"{v, k}"
way,30179,"{tag, nd}","{version, timestamp, uid, id, changeset, user}"
nd,405590,{},{ref}
relation,554,"{member, tag}","{version, timestamp, uid, id, changeset, user}"


The set of node attributes get cut off in the output:


{'changeset', 'id', 'lat', 'lon', 'timestamp', 'uid', 'user', 'version'}

### Tags
It's also worth noting that every top-level element uses tag subelements. Drilling down into the tags, where most of the cleaning needs to happen, we want to find as many problems as we can before loading the data into MongoDB. That way we can clean it as we write it to the JSON file.

If we were doing a comprehensive cleanup, we might divide the work among a team. We might want to get an idea of which tags will be quick wins and which will require more intensive investigation and cleanup. We could do that by getting a set of unique values for each key and getting a count of those unique values. There are hundreds of tag keys, some only used once or twice, and others used thousands of times. (See below.)

I'm neither doing a comprehensive cleanup, nor am I working on a team on this school project. I only need to clean a couple of tag types for this project. That said, I wanted to have fun and solve a few extra problems, so organizing the data into manageable, targetable chunks is useful.

In [3]:
tag_df

,tag_use_count,val_set,uniq_count,usage_per_uniq
tag_key,,,,
created_by,1836,"{Potlatch alpha, Potlatch 0.10b, polyshp2osm-m...",5,367.200000
highway,20660,"{street_lamp, secondary, road, raceway, cyclew...",39,529.743590
source,7414,{http://www.waymarking.com/waymarks/WMRJGA_Neu...,176,42.125000
name,11152,"{Tweedle Dee Tweedle Dum, Woodfern Way, Brandy...",6951,1.604373
ref,859,"{E, 11G, 87, 7G, MON 26, 190, MON 43, MON 5E-9...",204,4.210784
...,...,...,...,...
complete,1,{yes},1,1.000000
distance,1,{1200 miles},1,1.000000
education,1,{no},1,1.000000


<div class="alert alert-block alert-info">
In cases in which the dataset is too large to read the value sets into memory, the script could be rewritten to only get the element and tag lists and counts. That is, create the same dataframe without the value set column. Since you'd need to parse the XML to count each set, this would come at a considerable compute time cost. At that point, you might as well load it into a temporary database.
</div>

#### XML tag stats

In [4]:
tag_df.describe(percentiles=np.arange(start=.1, stop=1, step=.1))

,tag_use_count,uniq_count,usage_per_uniq
count,1032.000000,1032.000000,1032.000000
mean,127.791667,18.108527,14.431499
std,885.506474,231.232681,73.255911
min,1.000000,1.000000,1.000000
10%,1.000000,1.000000,1.000000
20%,1.000000,1.000000,1.000000
30%,2.000000,1.000000,1.000000
40%,2.000000,1.000000,1.000000
50%,3.000000,2.000000,1.000000
60%,3.000000,2.000000,1.500000


There are 1,032 unique tag keys. Most are used three or fewer times, and one is used 20,660 times, the most of any. Almost half of the tags only have a single unique value. The "name" key has the most unique values, 6,951, which makes it a good candidate for an index.

About 20% have two unique values. These are good candidates for boolean values, as are tags with a single unique value. However, many OSM tags with "yes" and "no" as possible values also include other possible values. We still need to take them each on a case-by-case basis, consulting the OSM wiki. I did this in some cases, but not all.

<a href="#contents">Back to contents.</a>
<a id="tagAudits"></a>
## Sample Tag Audits
### Phone audit
Say we're tasked with formatting all "phone" tags. First, we'll want to find what needs to be fixed, then we'll need to fix it.

We can start by laying eyes on the list. Uncomment and run the below code to see the list.

In [5]:
# # Uncomment to print full list.
# tag_df.loc['phone', 'val_set']

Scrolling through the list, we see that the biggest issue is that they are formatted in different ways using different separating characters, including or not including the country code, with some having extensions.

Additionally, at least one of these entries is two, semicolon-separated phone numbers. We'll need to audit listed values, too. But, I handle making this field a list elsewhere, so we'll just focus on formatting one phone number at a time here.

We want to check the area codes and length at least. A more rigorous audit might include finding all valid prefixes for each area code and validating the prefixes as well.

We can ignore the extensions for now and strip them off for the audit.

We'll start our script with that, then pull all of the non-digit characters out, make sure there are at least 10 digits long, and check the first four digits to make sure they contain the right area code and possibly country code. It doesn't have to have a country code. We'll take care of that in the cleanup script.

<div class="alert alert-block alert-info">
The assumption here is that there are no foreign numbers with a country code and area code that matches our local pattern (e.g. +13 60...), but I didn't see an instance of that, and it would be surprising to find an instance in this extract. A bigger extract might require a more robust audit.
</div>

In [6]:
first_four_re = re.compile(r'1*360')

def ex_audit_phone(num):
    # Could check other separators, but we know this is all we have.
    is_list = ";" in num
    if not is_list:
        # Strip extensions.
        # Not a clean strip, but removes numbers, which is all we care about.
        x_idx = num.find("x")
        if x_idx > -1:
            num = num[:(x_idx)]
        # Remove all non-digits.
        num = re.sub(r'\D', "", num)
        if len(num) >= 10 and first_four_re.match(num[:4]):
            num = None
    else:
        num_lst = num.split(";")
        num = list()
        for num_it in num_lst:
            num.append(ex_audit_phone(num_it))
            
    return num

for num in tag_df.loc['phone', 'val_set']:
    ex_audit_phone(num)

'3067562314'

'18882293770'

'12537097453'

'16045579901'

'16048567472'

...

'18444627342'

'18664558489'

'18663830777'

'13063988300'

'12064293813'

The lengths all look good. The only real problem with these is that they don't conform to the expected area code. The toll-free numbers are fine, as are the "253", "206", and "604" area codes, which are from a neighboring regions with the same country code. For our purposes, I'm going to say that "306" is a typo, not a Saskatchewan number, which is quite far away. They both have valid prefixes for Whatcom County (where Bellingham is) and for Saskatchewan, but you could try to verify with a phone call or lookup if you really needed to.

### Phone cleanup
The <a href='https://wiki.openstreetmap.org/wiki/Key:phone'>OSM wiki for the phone tag key</a> sets two standard formats:
```
phone=+<country code> <area code> <local number>
phone=+<country code>-<area code>-<local number>
```
We'll use the second format. We want to keep extensions, so we'll tack those on at the end:
```
phone=+<country code>-<area code>-<local number> x<extension>
```
We'll create a regular expression for the target format, find all numbers that don't match, remove all non-numerical characters, make sure there's a "+1" at the beginning, look for an extension, and insert separators where they belong.

We can use a quick and dirty regular expression for this extract.

In [7]:
PHONE_RE = re.compile(r'\+1-\d\d\d-\d\d\d-\d\d\d\d')
WRONG_AC_RE = re.compile(r'1*306')

number_lst = ["(360) 555-9999 ext. 1234", "+1 306-398-8300"]

def ex_format_phone(num):
    if not PHONE_RE.fullmatch(num):
        num = re.sub(r'\D', "", num)
        if WRONG_AC_RE.match(num):
            num = re.sub("306", "360", num, count=1)
        if num[0] != "1":
            num = "1" + num
        if len(num) > 12:
            num = num[:11] + " x" + num[11:]
        num = "+" + num[0] + "-" + num[1:4] + "-" + num[4:7] + "-" + num[7:]
        
    return num

for num in number_lst:
    ex_format_phone(num)

# for num in tag_df.loc['phone', 'val_set']:
#     ex_format_phone(num)

'+1-360-555-9999 x1234'

'+1-360-398-8300'

### Street audit
Now, say we're auditing street names in the "addr:street" tag. In addition to taking care of simple things like capitalization, part of the cleanup is to enforce a standard for street types. For instance, we want all avenues to be called "Avenue", not "Ave." or "Ave", or anything else.

We'll search for exceptions to acceptable street type values. Then, we can map those exceptions to the formatted string when we're running our cleanup and writing to a JSON file.

In [8]:
# # regex to grab the street name from the end of the string.
# street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# Acceptable street type formats.
street_type_lst = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place",
                   "Square", "Lane", "Road", "Trail", "Parkway", "Highway",
                   "Way"]
# Exceptions, street types not matching format.
street_type_exceptions_set = set()

for street in tag_df.loc["addr:street", "val_set"]:
#     # If using regex.
#     match = street_type_re.search(street)
#     if match:
#         street_type = match.group()
#         if street_type not in street_type_lst:
#             street_type_exceptions_set.add(street_type)

    # If not using regex, just split and grab.
    street_type = street.split()[-1]
    if street_type not in street_type_lst:
        street_type_exceptions_set.add(street_type)
        
street_type_exceptions_set

{'#101',
 '#215',
 '13',
 'Ave',
 'Ave.',
 'Bakerview',
 'Blvd',
 'Broadway',
 'Count',
 'Dr',
 'Forest',
 'Hwy',
 'Meridian',
 'North',
 'Pkwy',
 'Rd',
 'Rd.',
 'Road3',
 'St',
 'St.',
 'Street\\',
 'WA-542',
 'street'}

Some of these are street names missing suffixes. Some are abbreviations or typos, or something else. These can be easily added to the mapping dictionary below.

Broadway is actually a street in Bellingham without a suffix, so we can ignore it. We can ignore "WA-542" which is a highway.

Forest could be Forest Street or Forest Court. Bakerview could be Bakerview Road or Bakerview Spur. Meridian could be Meridian Street or Guide Meridian Road. We need to manually check those.

The numbers are probably unit numbers, but I want to make sure they aren't highway numbers or something else.

In [9]:
unhandled_lst = ["13", "#101", "#215", "Bakerview", "Count", "Forest",
                 "Meridian", "North", "Road3"]
unhandled_els = list()
for _, el in ET.iterparse(source=filename, events=('start',)):
    if el.tag == "tag" and el.attrib["k"] == "addr:street" \
        and el.attrib["v"].split()[-1] in unhandled_lst:
            unhandled_els.append(el)
for el in unhandled_els:
    print(el.attrib)

{'k': 'addr:street', 'v': 'North Forest'}
{'k': 'addr:street', 'v': 'Ellis St. #215'}
{'k': 'addr:street', 'v': 'West Bakerview'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Old Highway 99 North'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'East Holly Street, #101'}
{'k': 'addr:street', 'v': 'Highway 13'}
{'k': 'addr:street', 'v': 'Highway 13'}
{'k': 'addr:street', 'v': 'Chuckanut Drive North'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Guide Meridian'}
{'k': 'addr:street', 'v': 'Chuckanut Drive North'}
{'k': 'addr:s

There is no North Forest Court, so we know this is North Forest Street. Without the prefix, we would have had to find the node or way on OSM to be sure. Same with West Bakerview Road.

Colloquially, Guide Meridian Road is called Guide Meridian (or "the Guide" if you're a true local). Guide Meridian more properly refers to a district on the Guide. In the interest of being clear and consistent with other map sources (i.e. Google Maps), we'll add "Road".

Old Highway 99 North and Chuckanut Drive North are proper and can be ignored. Highway 13 can be ignored, but in another extract 13 might be an apartment number.

"Count" is a typo of "Court". "Road3" is a typo.

The numbers look like apartment or suite numbers. We can send those to ```"addr": {"unit": <unit number>}``` elsewhere in the code. But, we'll have to be careful not treat Highway 13 or WA-542 as unit numbers.
### Street cleanup
This cleanup is specific to this extract. A future extract of the same area will likely contain errors not handled by this code.

In [10]:
# Exceptions mapped to acceptable street type formats.
ex_STREET_TYPE_MAP = {"Ave": "Avenue", "Ave.": "Avenue", "Blvd": "Boulevard",
    "Bakerview": "Bakerview Road", "Count": "Court", "Dr": "Drive",
    "Forest": "Forest Street", "Hwy": "Highway", "Meridian": "Meridian Road",
    "Pkwy": "Parkway", "Rd": "Road", "Rd.": "Road", "Road3": "Road",
    "St": "Street", "St.": "Street", "Street,": "Street", "Street\\": "Street",
    "street": "Street"}

def ex_clean_street(street):
    unit = None
    street_type = street.split()[-1]
    if "#" in street_type:
        unit = street_type
        street = " ".join(street.split()[:-1])
        street_type = street.split()[-1]
    if street_type in ex_STREET_TYPE_MAP.keys():
         street = " ".join(street.split()[:-1]) + \
            " " + ex_STREET_TYPE_MAP[street_type]
            
    return street, unit

handled_lst = list()
for el in unhandled_els:
    handled_lst.append(ex_clean_street(el.attrib["v"]))
handled_lst

[('North Forest Street', None),
 ('Ellis Street', '#215'),
 ('West Bakerview Road', None),
 ('Guide Meridian Road', None),
 ('Old Highway 99 North', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('East Holly Street', '#101'),
 ('Highway 13', None),
 ('Highway 13', None),
 ('Chuckanut Drive North', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Guide Meridian Road', None),
 ('Chuckanut Drive North', None),
 ('Birch Bay Lynden Road', None),
 ('Meadowbrook Court', None)]

### Further auditing
Those are my two examples of showing my work as I audit the data, which is all that's required for this assignment, but I'm choosing to do more. I'll highlight some findings, but I won't show all of my work. See the imported scripts.

I'm not doing a comprehensive cleanup, but I decided to look at each and every tag because I want to familiarize myself with OSM, and with my dataset. I should note that, in the interest of time, I'm foregoing a rigorous audit from here, having satisfied the requirements of the assignment, so I will likely make some mistakes that I wouldn't otherwise in a full audit.

I printed out all of the tag keys with their value sets and looked them over. I skimmed over tags that seemed pretty self-evident and didn't have any troublesome values that caught my eye. I checked the OSM wiki for the standards on the tags that did catch my eye.

In [11]:
# # Let's dig in!
# # Uncomment the following block to print out all of the tag keys with their
# # value sets.

# for tag_key, tag_val_set in sorted(tag_df['val_set'].items()):
#     (tag_key, tag_val_set)

<a href="#contents">Back to contents.</a>
<a id="trouble"></a>
## Trouble in the Data
### Structural trouble
A few things quickly stand out. Some of them have to do with restructuring XML into JSON.

#### Colon-separated keys

For instance, many keys have colon-separated values that represent a hierarchical structure (e.g. "addr:street=" and "addr:state="). These keys can be subdivided into lower-order documents. But, not all keys like this can be subdivided so easily. Some of the parent prefixes are also keys that stand alone and have their own values (e.g. "turn=", "turn:lane="). We would need to create a child key identical to the parent key. This might have undesirable effects. That said, this might be a moot point if we were to group tags by parent elements. Since I have no analytical or use goals for this database, I chose to only subdivide a select list of keys in which the parents never stand alone: addr, cost, fire_hydrant, fuel, payment.

#### Lists

We also need to decide which tags have list values and which have scalar values. A starting point might be to identify those with semicolon-separated values by searching for values with semicolons. If you were to programmatically search for semicolons, you'd still need to look manually in those values that might employ a semicolon as natural language rather than as a separator, such as in "note" and "description" tags. Plus, what about those tags that warrant a list but happen to only have a scalar value in this extract? Rather than developing (or finding) an AI for this task, I decided to put eyes on each tag, consulting the wiki as necessary, and make a list by hand. That said, I did run a search for semicolons to catch some tags that I might have missed. Again, I'm only doing this part casually to create some problems to solve, beyond what's required.

Other candidates for creating list values are those tags with keys that end in an underscore and a number, indicating multiple values for the same type of tag (e.g. 'name_1'). I compiled this list programmatically.

The "is_in" key and all of its children is another quick check. We could omit it since this extract is exlcusive to Bellingham, WA. But, hypothetically, this data might be joined to other extracts, so I left it in and mapped it to standard value strings.

Any key with a "conditional" suffix has a list.

#### Other
    
The "contact" prefix is deprecated, so I removed it. These keys should hold list values.
    
Finally, I substituted some miscellaneous keys for others, due to deprecation, misuse, redundancy, etc. However, I'm choosing to leave the "tiger", "gnis", and "nist" tags alone in lieu of forming a broader plan for them. I did fix a couple of gnis typos.

I do recommend replacing gnis:ST_num: with gnis:state_id and gnis:County_num: with gnis:county_id because they are more widely used, but I left them.

### Trouble with values
The rest of the trouble is basically copy editing values. For instance, addresses, phone numbers, dates, etc. need to be formatted.

#### Geographical scope

Some elements point to locations outside of the map, in nearby towns: Ferndale, Lawrence, Eliza Island, Sudden Valley, Nooksack River; "addr:postcode" 98248 is Ferndale, 98262 is Lummi Island, 98266 is Maple Falls, 98284 is Skagit County which is neighboring but may also be a typo for Ferndale's zip. These might be legitimate inclusions. Wikipedia references with outside locations might be legitimate, for instance. They may point to the Bellingham page from the pages of nearby locations. I waited to investigate them once I'd loaded them into MongoDB.

#### Data types

I stored some values as numbers, but some of them have units tacked on or are otherwise messy. These mostly require a decision about whether to subdivide by unit, convert into a single unit, or leave as strings. I'm leaving them as strings to limit my scope of cleanup.

#### Symbol and image links

Multiple "symbol" tags link to an image at wikimedia.org. The "symbol" tag is for human-readable descriptions, not links. According to the OSM wiki, these links would be better placed in a "wiki:symbol" tag. There's some question about the proper way to link to images in the "wiki:symbol" tag. The OSM wiki entry on the tag states that these should only be filenames of files uploaded to the OSM wiki, not to external sites. No examples of this method exist in this extract, though. Most of the links do have identical filenames to identical symbols on the OSM wiki, so they could easily be stripped of their paths. However, one link does not and needs to be uploaded to OSM. I might do that once I gain that account privilege. In the meantime, a few forum users have stated, contrary to the OSM wiki article, that a link to wikimedia is fine. So, I just assigned the links to the appropriate key.

#### The rest

I found a long list of other copy edits to make, typos and formatting issues mostly. Some of which I handled. See the cleanup script for a commented list of what I left unhandled.

I also created a laundry list of stickier issues that aren't simply edits to be made, and that I will not likely fix. They may require a FIXME tag in the OSM data, or an executive/team decision with regard for the intended use of the data.

For instance, the OSM wiki doesn't include the "wheelchair:entrance_width" and "wheelchair:step_height" tags. They therefore have no default unit, and the tags in the extract don't give units. OSM defaults to metric units, but this is a U.S. dataset, so the contributor may have used imperial units. This warrants issuing a FIXME tag in the element, and perhaps creating a page on the wiki if no sufficient alternative tag exists. It also may fit nicely into the following category.

#### Field trips and curiosities

I collected some cases of what I'm calling "field trips and curiosities." They include unexpected values that beckon a closer look at the data, if not also an investigation "in real life" (e.g. "width": "Cedar Jumps Green Line", "maxspeed": "127", "start_date": "0000"). Some of them just need a phone call for business hours or a bike ride with a tape measure. Data wrangling sometimes requires boots on the ground to establish ground truth.

Alright, let's get to cleaning up!

<a href="#contents">Back to contents.</a>
<a id="load"></a>
## Cleanup, Write to JSON, Load into MongoDB
Again, we want to clean the data as we write it to a JSON file, rather than after it's already loaded to MongoDB. So, we need functions that handle each fix while parsing through the XML.

Below, we reshape the XML into JSON-compatible Python dictionaries and lists, representing the three top-level OSM elements. Then we write those as documents to a JSON file and import the file into a MongoDB collection.

In [12]:
if os.path.exists("bham.json"):
    !del bham.json

clean_and_write.process_map(file_in=filename, fo_pre="bham", pretty=False)

In [13]:
mongo_client = MongoClient("localhost:27017")
osm_db = mongo_client.osm
bham_col = osm_db.bham

In [14]:
mongo_client.drop_database("osm")
# !"C:\Program Files\MongoDB\Tools\100\bin\mongoimport.exe"
!mongoimport -d osm -c bham --file bham.json

2021-04-03T09:19:09.285-0700	connected to: mongodb://localhost/
2021-04-03T09:19:12.286-0700	[########................] osm.bham	40.3MB/109MB (36.8%)
2021-04-03T09:19:15.286-0700	[##################......] osm.bham	83.3MB/109MB (76.1%)
2021-04-03T09:19:16.959-0700	[########################] osm.bham	109MB/109MB (100.0%)
2021-04-03T09:19:16.959-0700	385777 document(s) imported successfully. 0 document(s) failed to import.


<a href="#contents">Back to contents.</a>
<a id="mongoStats"></a>
## Import and MongoDB Stats
The XML document is almost 77 MB, and grew to a JSON file about 115 MB big. After import, the new database total size (storage plus indexes) is 1.6 MB. However, due to compression, its data size is larger, 88.5 MB.

There are 355,044 nodes, 30179 ways, and 354 relations. And, there are 921 unique users.

In [15]:
os.stat("greater_bellingham.osm").st_size

76832450

In [16]:
os.stat("bham.json").st_size

114778234

In [17]:
osm_db.command("dbstats")

{'db': 'osm',
 'collections': 1,
 'views': 0,
 'objects': 385777,
 'avgObjSize': 229.51770064052548,
 'dataSize': 88542650.0,
 'storageSize': 4096.0,
 'indexes': 1,
 'indexSize': 1622016.0,
 'totalSize': 1626112.0,
 'scaleFactor': 1.0,
 'fsUsedSize': 97910169600.0,
 'fsTotalSize': 255465693184.0,
 'ok': 1.0}

In [18]:
# Count all documents by document type.
doc_count_lst = mongo_audit.check_doc_counts_by(coll=bham_col, count_k="_id",
                                doc_type_lst=["node", "way", "relation"],
                                group_k="doc_type")
print("Keys per document type:")
doc_count_lst
for count in doc_count_lst:
    print("Pre-load OSM", count["_id"], "count matches MongoDB count?")
    print(count["count"] == el_df.loc[count["_id"], "count"])

Keys per document type:


[{'_id': 'node', 'count': 355044},
 {'_id': 'way', 'count': 30179},
 {'_id': 'relation', 'count': 554}]

Pre-load OSM node count matches MongoDB count?
True
Pre-load OSM way count matches MongoDB count?
True
Pre-load OSM relation count matches MongoDB count?
True


In [19]:
list_query(mongo_audit.get_unique_users(bham_col))

[{'unique_users': 921}]

<a href="#contents">Back to contents.</a>
<a id="mongoAudit"></a>
## Auditing in Mongo
### Updating values
Something I noticed while working with the data were a lot of addresses lacking states. I decided to fix them now that I can query.

I'm guessing every address should be in Washington State. This was an accurate statement when I used a map that I didn't draw by hand overlapping into British Columbia; we'll pretend it's true here.

I don't want to corrupt the data, so I will only set the state for addresses with zip codes. If I wanted to attack this problem with a map that included BC and WA, I might get lists of BC and WA zip codes and map the new state value accordingly.

In [20]:
mongo_audit.update_states(bham_col)

Updating states:


Address Key,Zip,State,Address
Count,,,
Pre_update,1103.0,689.0,1749.0
Matched,1103.0,0.0,0.0
Modified,0.0,546.0,0.0
Post_update,1103.0,1234.0,1749.0


### Structural checks
Let's see if tag keys were subdivided properly. I'm curious about bicycle services specifically. So, I'll check the services tag.

In [21]:
# Count service documents.
mongo_audit.check_doc_counts_by(coll=bham_col, count_k="service",
                                   doc_type_lst=["node", "way", "relation"],
                                   group_k="doc_type")

[{'_id': 'node', 'count': 13},
 {'_id': 'way', 'count': 3529},
 {'_id': 'relation', 'count': 1}]

In [22]:
mongo_audit.get_bike_services(coll=bham_col)

Documents with bicycle services, shops, and/or bike repair stations:

[{'_id': '255801449', 'name': ['Fairhaven Bike and Ski'], 'shop': ['bicycle']},
 {'_id': '255801452',
  'name': ['The Hub'],
  'service': {'bicycle': {'diy': 'yes', 'second_hand': 'yes'}},
  'shop': ['bicycle']},
 {'_id': '1976759477', 'name': ['Trek Cycles'], 'shop': ['bicycle']},
 {'_id': '3191111061', 'name': ['Fanatik Bike Co.'], 'shop': ['bicycle']},
 {'_id': '3497698417',
  'amenity': ['bicycle_repair_station'],
  'note': 'Western Washington University',
  'opening_hours': ['24/7'],
  'service': {'bicycle': {'chain_tool': 'no'}}},
 {'_id': '4049510936', 'amenity': ['bicycle_repair_station']},
 {'_id': '4049598206',
  'amenity': ['bicycle_repair_station'],
  'note': 'Wade King Rec Center',
  'opening_hours': ['24/7'],
  'service': {'bicycle': {'pump': 'yes'}}},
 {'_id': '4049598207', 'amenity': ['bicycle_repair_station']},
 {'_id': '4176487913',
  'amenity': ['bicycle_repair_station'],
  'note': 'Haggard Hall',


It looks like the service key was properly subdivided, and we have several bicycle services recorded in town.

It's surprising that there's a relation with a service key. I would expect service keys specifying the types of services a shop offers (e.g. "service:bicycle..." to belong to nodes, while service keys with a single value (e.g. "service=parking_aisle") to describe a way.

But, I validated documents in the cleanup script before writing to JSON. All node documents include a position, and do not include node references nor members. All way documents include node references but neither a position nor members. All relation documents include members, no position, and no node references.

So, as long as the documents are properly formed, it's okay for services to be expressed as relations. I did find the three unexpectedly classified services, and they were vehicle repair and lube shops, where it might make sense to document the shop area or flow of the driveway.

<a href="#contents">Back to contents.</a>
<a id="finalQ"></a>
## Final Queries: using lookup; another audit; indexes
For my final query, I want to find which users contributed the most-referenced service documents. That is, which service documents are referenced most, and who created them?

In the process of answering this question, I found a problematic error in the data. Some documents reference other documents but classify them as the wrong type of documents. This disrupted the NoSQL equivalent of referential integrity when unwinding and regrouping to answer the question, which posed a problem when trying to write a "join" collection.

So, before answering my question, I need to make sure all references to other documents agree on the document type.

Once that is fixed, I create a "join" collection of referenced documents with a list of their referencing documents. This helps find the most-referenced service documents, and it would be generally useful for some hypothetical further queries.

From there, I just sort by most-referenced, then look up which documents are service documents and grab their users.

### Auditing references

In [23]:
mongo_audit.audit_ref_types(coll=bham_col, coll_str="bham")

Ways point to the following types:
[{'type': ['node']}]
Relations point to the following types, and refer to them as:
[{'referred_as': ['way', 'node'], 'type': ['node']},
 {'referred_as': ['relation'], 'type': ['relation']},
 {'referred_as': ['way'], 'type': ['way']}]


In [24]:
mismatched_members_lst = mongo_audit.\
                            get_doctype_mismatches(coll=bham_col,
                                                   coll_str="bham")

Ways pointing to non_nodes:


Relations with mismatched referenced document types:
[{'_id': '2317217',
  'members': {'ref': '37125674', 'role': 'forward', 'type': 'node'},
  'refs': {'_id': '37125674', 'doc_type': 'node'}},
 {'_id': '2859142',
  'members': {'ref': '37125674', 'role': '', 'type': 'node'},
  'refs': {'_id': '37125674', 'doc_type': 'node'}}]


In [25]:
mongo_audit.\
 fix_mismatched_refs(coll=bham_col,
                     mismatched_members_lst=mismatched_members_lst)

Member to update:
{'_id': '2317217', 'members': [{'type': 'way', 'ref': '37125674', 'role': 'forward'}]}
Updated member:
{'_id': '2317217', 'members': [{'type': 'node', 'ref': '37125674', 'role': 'forward'}]}
Referenced document:
{'_id': '37125674', 'doc_type': 'node'} 

Member to update:
{'_id': '2859142', 'members': [{'type': 'way', 'ref': '37125674', 'role': ''}]}
Updated member:
{'_id': '2859142', 'members': [{'type': 'node', 'ref': '37125674', 'role': ''}]}
Referenced document:
{'_id': '37125674', 'doc_type': 'node'} 



### Creating reference table and using it
Okay, now we can create our reference lookup table and use it to find out who contributed the most-referenced documents.

In [26]:
ref_docs_col = mongo_audit.write_ref_docs(db=osm_db, coll=bham_col)

print("Sample ref_docs. id is referenced doc; refers are referencing docs:")
list_query(ref_docs_col.find().limit(1))

Sample ref_docs. id is referenced doc; refers are referencing docs:


[{'_id': '7311005915', 'refers': ['782885150']}]

In [27]:
# Which service documents are referenced most, and who contributed them?
list_query(mongo_audit.get_most_refd(coll=bham_col, field="service", limit=3))

[{'_id': '495336835',
  'refer_count': 6,
  'contributor': ['DunbarLoop'],
  'contributer_uid': ['2032374']},
 {'_id': '273637364',
  'refer_count': 6,
  'contributor': ['DunbarLoop'],
  'contributer_uid': ['2032374']},
 {'_id': '273637353',
  'refer_count': 6,
  'contributor': ['DunbarLoop'],
  'contributer_uid': ['2032374']}]

### Adding indexes
And, we might as well toss in a couple of indexes for hypothetical future use.

We established earlier that name is a good candidate, since it has the highest cardinality. We'll make it a sparse index since not every document has a name.

And, let's make a geospacial index on the coordinates field ("pos", should only be on nodes). We can't make it a unique index since contributors have added nodes with identical coordinates to other nodes. This could be an area for future cleanup, or it could be the result of a necessary hack (e.g. enabling identical coordinates to be tagged with multiple mutually exclusive tags).

In [28]:
bham_col.create_index([("name", pymongo.DESCENDING)], sparse=True)
bham_col.create_index([("pos", pymongo.GEO2D)])

'name_-1'

'pos_2d'

<a href="#contents">Back to contents</a>
<a id="recommendation"></a>
## A Parting Recommendation
Disambiguate units of measurement and save as numbers to increase the utility of the database. This will enable calculations of all sorts, like calculating differences in elevation or finding the minimum maxheight on a route.

Many users undoubtedly follow OSM standard and use metric units, but many contributors have used imperial units in measurements such as maxheight, as evidenced by the use of single and double quotes in the values indicating feet and inches. Those measurements are fairly easy to find and fix, but what of those measurements with no explicit unit? Do they follow the metric system or imperial?

It would likely be necessary to reference outside sources, such as known bridge heights from the Department of Transportation.

In [29]:
list_query(mongo_audit.get_by_field(coll=bham_col, field="maxheight"))

[{'_id': '538611921', 'maxheight': 0.0},
 {'_id': '665129111', 'maxheight': '13"1\''},
 {'_id': '665129114', 'maxheight': '13"1\''},
 {'_id': '665669883', 'maxheight': '9\'10"'},
 {'_id': '814346764', 'maxheight': 3.55}]